In [15]:
#출처 : https://ross.readthedocs.io/en/stable/user_guide/example_13.html
# pip install ross-rotordynamics

import ross as rs
import ross.stochastic as srs
import numpy as np
import plotly.graph_objects as go

# uncomment the lines below if you are having problems with plots not showing
# import plotly.io as pio
# pio.renderers.default = "notebook"

In [16]:
# Deterministic Shaft Elements
# Steel = rs.steel   # 에러 발생

# 물성치 직접 입력
Steel = rs.Material("steel", E=211e9, G_s=81.2e9, rho=7810)

# Shaft Dimension 입력
shaft = [rs.ShaftElement(L=0.25, material=Steel, idl=0, odl=0.05) for i in range(6)]

# 두 개의 디스크 입력
# Deterministic Disk Elements
disk1 = rs.DiskElement.from_geometry(
    n=2,
    material=Steel,
    width=0.07,
    i_d=0.05,
    o_d=0.28,
)

disk2 = rs.DiskElement.from_geometry(
    n=4,
    material=Steel,
    width=0.07,
    i_d=0.05,
    o_d=0.35,
)
disks = [disk1, disk2]

In [35]:
# Stochastic(확률론적) Bearing Element

# random variables must have the same size
kxx = np.random.uniform(low=1e6, high=3e6, size=5)
cxx = np.random.uniform(low=1e2, high=2e2, size=5)

bearing1 = srs.ST_BearingElement(n=0, kxx=kxx, cxx=cxx, is_random=["kxx", "cxx"])
bearing2 = srs.ST_BearingElement(n=6, kxx=kxx, cxx=cxx, is_random=["kxx", "cxx"])
bearings = [bearing1, bearing2]

In [36]:
print(cxx)
print(kxx)

[104.96072848 144.25689992 110.53375109 154.04457741 159.17473014]
[1446117.58456076 1724073.94571761 1896347.36637783 2052546.95621947
 1131168.9381669 ]


In [37]:
# Building random instances for a rotor model

rand_rotor = srs.ST_Rotor(
    shaft_elements=shaft, disk_elements=disks, bearing_elements=bearings
)

# Number of samples
print("Number of samples:", len(list(iter(rand_rotor))))

Number of samples: 5


In [38]:
sample = np.random.choice(list(iter(rand_rotor)))
sample.plot_rotor()

In [39]:
# Running Stochastic Campbell Diagram

speed_range = np.linspace(0, 600, 31)
camp = rand_rotor.run_campbell(speed_range)

In [40]:
# Plotting Stochastic Campbell Diagram

# choose the desirable percentiles or confidence intervals
fig1 = camp.plot_nat_freq(conf_interval=[95], width=950, height=700)
fig1.show()

In [41]:
# choose the desirable percentiles or confidence intervals
fig2 = camp.plot_log_dec(conf_interval=[95], width=950, height=700)
fig2.show()

In [64]:
Q_ = rs.Q_
modal = sample.run_modal(speed=Q_(100, "RPM"))   # Speed를 변경했을 때 해석 결과 - 모드 형상 - 의미? (241106)

In [65]:
for mode in range(6):
    display(modal.plot_mode_3d(mode, frequency_units="Hz"))